In [ ]:
# Install ray version without device bugs
!pip install -U ray[default]==2.0 > /dev/null 2>&1
!pip install -U ray[rllib]==2.0 > /dev/null 2>&1
# Install other dependencies
!pip install pygame > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U colabgymrender > /dev/null 2>&1
!pip install imageio==2.4.1 > /dev/null 2>&1

# Install gym
!pip install gym[atari] > /dev/null 2>&1
!pip install gym[accept-rom-license] > /dev/null 2>&1
#!pip install gym[accept-rom-license]

# GPU monitoring
!pip install GPUtil > /dev/null 2>&1

In [ ]:
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

<Surface(640x480x32 SW)>

In [ ]:
import ray
from ray import air, tune

ray.init()

2022-11-05 13:30:20,394	INFO worker.py:1515 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.7.15
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8265


In [ ]:
from gym import envs
import gym
#print(envs.registry.all())

In [ ]:
env_name = "MsPacman-v4"
from ray.rllib.env.wrappers.atari_wrappers import WarpFrame, MaxAndSkipEnv, FrameStack
dim = 84

class AtariWapped(gym.Env):
    def __init__(self, env_config={}):
        self.env = gym.make(env_name, **env_config)
        self.env = WarpFrame(self.env, dim=dim)
        self.env = MaxAndSkipEnv(self.env, skip=4)
        self.env = FrameStack(self.env, k=4)
        #env = gym.make(env_name, frameskip=0)
        #self.env = gym.wrappers.AtariPreprocessing(env, noop_max=30, frame_skip=16, screen_size=dim, terminal_on_life_loss=True, grayscale_obs=True, grayscale_newaxis=False, scale_obs=False)
        self.action_space = self.env.action_space
        self.observation_space = self.env.observation_space

    def step(self, action):
        return self.env.step(action)

    def reset(self):
        return self.env.reset()

    def render(self, *args, **kwargs):
        return self.env.render(*args, **kwargs)

e_info =  AtariWapped()

from copy import deepcopy

config = deepcopy({
  'observation_space': e_info.observation_space,
  'action_space': e_info.action_space
})

wrapped_name = AtariWapped
from ray.tune.registry import register_env

def env_creator(env_config):
    return AtariWapped(env_config)
final_env_name = "attari_wrapped"
register_env(final_env_name, env_creator)


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:506: UserWarning: WARN: The environment MsPacman-v4 is out of date. You should consider upgrading to version `v5` with the environment ID `ALE/MsPacman-v5`.
  f"The environment {path} is out of date. You should consider "
2022-11-05 13:30:27,804	WARNING deprecation.py:48 -- DeprecationWarning: `FrameStack` has been deprecated. This will raise an error in the future!


In [ ]:
AtariWapped().reset().shape

(84, 84, 4)

In [ ]:
from ray.rllib.algorithms.dqn import DQN
from ray.rllib.algorithms.ppo import PPO
from math import sqrt

In [ ]:
from ray.tune.stopper import Stopper, TrialPlateauStopper, CombinedStopper, MaximumIterationStopper


# Stop on degradation
es = CombinedStopper(TrialPlateauStopper(metric='episode_reward_mean', num_results = 4, mode = 'max', grace_period = 4), MaximumIterationStopper(max_iter=2000))

In [ ]:
from ray.tune.schedulers import HyperBandScheduler

sc = HyperBandScheduler('time_total_s', metric='episode_reward_mean', max_t=1800, mode="max")

In [ ]:
import psutil

eps_n_workes = 0.001

In [ ]:
Algo = DQN
save_per_training_iteration = 10
train_batch_size = 512
num_workers = 30#100
num_av_cpus = psutil.cpu_count() - 1

# run on CPU
workers_gpu_frac = 0
num_gpus_per_worker = (1/num_workers)*workers_gpu_frac 

lr = [1e-4] #[1e-5, 1e-4]#[0.01, 1e-3, 1e-4, 1e-5]
config = {
        "env": final_env_name,
        "num_gpus": 1-workers_gpu_frac,
        "num_gpus_per_worker": num_gpus_per_worker,#(1/num_workers)*workers_gpu_frac,
        "num_cpus_per_worker": (1/num_workers)*num_av_cpus, 
        "num_workers": num_workers,
        "lr": tune.grid_search(lr),
        "framework": "torch",
        #'evaluation_num_workers': 0,
        #'evaluation_parallel_to_training': False,
        "train_batch_size": train_batch_size,
        #'num_rollout_workers': 1,
        'soft_horizon': False,
        'horizon': 100000,
  }

# if type(Algo) == DQN:
#   config["optimizer"] = "ADAM"

if type(Algo) == PPO:
  # Like in paper
  config['grad_clip'] = 0.2
  config['horizon'] = 128
  config["optimizer"] = "ADAM"
  #config["train_batch_size"] = 32*num_workers
  lr = [1e-4*sqrt(num_workers)]
  

results = tune.Tuner(
    Algo,
    tune_config=tune.TuneConfig(scheduler=sc),
    run_config=air.RunConfig(
        #stop=es,#{"training_iteration": training_iteration, },
        checkpoint_config=air.CheckpointConfig(checkpoint_frequency=save_per_training_iteration),
        name="example-experiment-atari",
        local_dir="./example-experiment"
        ),
    param_space=config,
).fit()

Trial name,status,loc,lr,iter,total time (s),ts,reward,num_recreated_wor...,episode_reward_max,episode_reward_min
DQN_attari_wrapped_7b891_00000,TERMINATED,172.28.0.2:221596,0.0001,378,1801.18,408240,1402.3,0,3760,320


  ).remote(self)

  stacklevel=1,

  return actor_cls._remote(args=args, kwargs=kwargs, **updated_options)

  return actor_cls._remote(args=args, kwargs=kwargs, **updated_options)

  return actor_cls._remote(args=args, kwargs=kwargs, **updated_options)

(pid=221596) /usr/local/lib/python3.7/dist-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(pid=221596)   import imp
(pid=221596) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow/python/util/deprecation.py:561: calling function (from tensorflow.python.eager.def_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=221596) Instructions for updating:
(pid=221596) experimental_relax_shapes is deprecated, use reduce_retracing instead
(pid=221596) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow/python/util/deprecation.py:561: calling f

Streaming output truncated to the last 5000 lines.
  iterations_since_restore: 345
  node_ip: 172.28.0.2
  num_agent_steps_sampled: 372600
  num_agent_steps_trained: 1585664
  num_env_steps_sampled: 372600
  num_env_steps_sampled_this_iter: 1080
  num_env_steps_trained: 1585664
  num_env_steps_trained_this_iter: 4608
  num_faulty_episodes: 0
  num_healthy_workers: 30
  num_recreated_workers: 0
  num_steps_trained_this_iter: 4608
  perf:
    cpu_util_percent: 48.775000000000006
    gpu_util_percent0: 0.0
    ram_util_percent: 21.3
    vram_util_percent0: 0.06320307874481942
  pid: 221596
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf:
    mean_action_processing_ms: 0.2968391892148962
    mean_env_render_ms: 0.0
    mean_env_wait_ms: 10.210849971777122
    mean_inference_ms: 21.13665929661119
    mean_raw_obs_processing_ms: 1.2689296690518876
  sampler_results:
    custom_metrics: {}
    episode_len_mean: 243.14
    episode_media: {}
    episode_r

2022-11-05 19:36:04,768	INFO tune.py:759 -- Total run time: 1977.74 seconds (1976.83 seconds for the tuning loop).


In [ ]:
from ray.tune import ExperimentAnalysis
analysis = ExperimentAnalysis("/content/example-experiment/")

2022-11-05 19:36:05,196	INFO experiment_analysis.py:796 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.


In [ ]:
trial_logdir = analysis.get_best_logdir(metric="episode_reward_mean", mode="max")  # Can also just specify trial dir directly

checkpoints = analysis.get_trial_checkpoints_paths(trial_logdir)  # Returns tuples of (logdir, metric)
best_checkpoint = analysis.get_best_checkpoint(trial_logdir, metric="episode_reward_mean", mode="max")

In [ ]:
agent = Algo(config={"framework": "torch"}, env=final_env_name)
agent.restore(best_checkpoint)

2022-11-05 19:36:05,428	WARNING deprecation.py:48 -- DeprecationWarning: `config['multiagent']['replay_mode']` has been deprecated. config['replay_buffer_config']['replay_mode'] This will raise an error in the future!
  stacklevel=1,

  f"The environment {path} is out of date. You should consider "

2022-11-05 19:36:05,792	INFO trainable.py:669 -- Restored on 172.28.0.2 from checkpoint: /content/example-experiment/example-experiment-atari/DQN_attari_wrapped_7b891_00000_0_lr=0.0001_2022-11-05_19-03-08/checkpoint_000240
2022-11-05 19:36:05,794	INFO trainable.py:677 -- Current state after restoring: {'_iteration': 240, '_timesteps_total': None, '_time_total': 1130.0314576625824, '_episodes_total': 1171}


In [ ]:
from colabgymrender.recorder import Recorder

video_every = 1
env = AtariWapped()
env = Recorder(env, "./video", 0.2)

  f"The environment {path} is out of date. You should consider "



In [ ]:
# run until episode ends
episode_reward = 0
done = False
obs = env.reset()
while not done:
    action = agent.compute_action(obs)
    obs, reward, done, info = env.step(action)
    episode_reward += reward

print(episode_reward)

2490.0
